[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eliewolfe/d-separation/blob/main/igraph_playground.ipynb)

In [1]:
import numpy as np
from scipy import sparse
from igraph import *

In [2]:
#import pip
#pip.main(['install','--upgrade','python-igraph'])

In [3]:
#Here are some useful igraph commands:
#get_inclist: can be used to obtain a vertex's parents or children
#subcomponent: can be used to obtain a vertex's ancestors or descendants
#all_st_mincuts: find screen off sets
#all_minimal_st_separators()  find screen off sets, made not be useful output
#minimum_size_separators() not sure what the difference is with previous, only works on undirected

#TOWARD IMPLEMENTING D-SEPERATION TEST
#induced_subgraph
#is_connected('WEAK')
#as_undirected()
#subcomponent(v,'ALL') #for a node
#get_subisomorphisms_vf2
#get_subisomorphisms_lad

In [4]:
#it may be much easier to use graph-tools https://graph-tool.skewed.de/static/doc/index.html, not on Windows
#or networkx

In [5]:
#Graph([(0,1), (0,2), (2,3), (3,4), (4,2), (2,5), (5,0), (6,3), (5,6)])

In [6]:
def ToTopologicalOrdering(g):
    return g.permute_vertices(np.argsort(g.topological_sorting('out')).tolist())

def ToCanonicalOrdering(g):
    canonical_ordering=np.argsort(np.flip(np.argsort(g.canonical_permutation()))).tolist() #argsorted and flipped and argsorted to maintain topological ordering. Not critical, but why not.
    return g.permute_vertices(canonical_ordering)

def ToSparseRepresentation(g):
    return sparse.lil_matrix(g.get_adjacency_sparse())

In [7]:
def LearnParametersFromGraph(origgraph):
    g=ToTopologicalOrdering(origgraph)
    verts=g.vs
    verts["parents"]=g.get_adjlist('in');
    verts["children"]=g.get_adjlist('out');
    verts["ancestors"]=[g.subcomponent(i,'in') for i in g.vs]
    verts["descendants"]=[g.subcomponent(i,'out') for i in g.vs]
    verts["indegree"]=g.indegree()
    verts["outdegree"]=g.outdegree() #Not needed
    verts["grandparents"]=g.neighborhood(None, order=2, mode='in', mindist=2)
    verts["parents_inclusive"]=g.neighborhood(None, order=1, mode='in', mindist=0) #Not needed
    has_grandparents=[idx for idx,v in enumerate(g.vs["grandparents"]) if len(v)>=1]
    verts["isroot"]=[0==i for i in g.vs["indegree"]]
    root_vertices=verts.select(isroot = True).indices
    nonroot_vertices=verts.select(isroot = False).indices
    latent_count=len(root_vertices) #Not needed
    verts["roots_of"]=[np.intersect1d(anc,root_vertices).tolist() for anc in g.vs["ancestors"]]
    def FindScreeningOffSet(root,observed):
        screeningset=np.intersect1d(root["descendants"],observed["parents"]).tolist()
        screeningset.append(observed.index)
        return screeningset
    determinism_checks=[(root,FindScreeningOffSet(verts[root],v)) for v in g.vs[has_grandparents] for root in np.setdiff1d(v["roots_of"],v["parents"])]
    return verts["name"],verts["parents"],verts["roots_of"],determinism_checks

In [8]:
g=Graph.Formula("U3->A:C:D,U2->B:C:D,U1->A:B,A->C,B->D")
names,parents_of,roots_of,determinism_checks = LearnParametersFromGraph(g)
print(names)
[parents_of,roots_of,determinism_checks]

['U3', 'U2', 'U1', 'A', 'B', 'C', 'D']


[[[], [], [], [0, 2], [1, 2], [0, 1, 3], [0, 1, 4]],
 [[0], [1], [2], [0, 2], [1, 2], [0, 1, 2], [0, 1, 2]],
 [(2, [3, 5]), (2, [4, 6])]]

In [9]:
g.get_isomorphisms_vf2()

[[0, 1, 2, 3, 4, 5, 6], [4, 5, 3, 2, 0, 1, 6]]

In [12]:
print(g.summary(1))

IGRAPH DN-- 7 10 -- 
+ attr: name (v)
+ edges (vertex names):
U3->A, U3->C, U3->D, A->C, U2->C, U2->D, U2->B, B->D, U1->A, U1->B


In [13]:
~np.any(sparse.csr_matrix.toarray(ToSparseRepresentation(ToTopologicalOrdering(g))!=ToSparseRepresentation(ToCanonicalOrdering(g))))

True

In [14]:
ToSparseRepresentation(ToCanonicalOrdering(g)).toarray()

array([[0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [15]:
g=Graph.Formula("U1->X,X->A,A->B,U2->A:B")
names,parents_of,roots_of,determinism_checks = LearnParametersFromGraph(g)
print(names)
[parents_of,roots_of,determinism_checks]

['U1', 'U2', 'X', 'A', 'B']


[[[], [], [0], [1, 2], [1, 3]],
 [[0], [1], [0], [0, 1], [0, 1]],
 [(0, [2, 3]), (0, [3, 4])]]

In [16]:
g=Graph.Formula("U1->A:C,U2:B:C:D,U3->A:D,A->B,B-->D")
names,parents_of,roots_of,determinism_checks = LearnParametersFromGraph(g)
print(names)
[parents_of,roots_of,determinism_checks]

['U1', 'U2', 'U3', 'C', 'A', 'B', 'D']


[[[], [], [], [0], [0, 2], [4], [2, 5]],
 [[0], [1], [2], [0], [0, 2], [0, 2], [0, 2]],
 [(0, [4, 5]), (2, [4, 5]), (0, [5, 6])]]

In [17]:
g=Graph.Formula("U1->A:C,U2:B:D,U3->A:D,A->B->C->D")
names,parents_of,roots_of,determinism_checks = LearnParametersFromGraph(g)
print(names)
print(g.get_edgelist())
[parents_of,roots_of,determinism_checks]

['U1', 'U2', 'U3', 'A', 'B', 'C', 'D']
[(0, 1), (0, 2), (1, 4), (2, 5), (4, 2), (6, 1), (6, 5)]


[[[], [], [], [0, 2], [3], [0, 4], [2, 5]],
 [[0], [1], [2], [0, 2], [0, 2], [0, 2], [0, 2]],
 [(0, [3, 4]), (2, [3, 4]), (2, [4, 5]), (0, [5, 6])]]

In [18]:
lst=[v["name"] for e in g.es for v in [*e.vertex_tuple]]
print([lst[i:(i+2)] for i in range(0, len(lst), 2)])

[['U1', 'A'], ['U1', 'C'], ['A', 'B'], ['C', 'D'], ['B', 'C'], ['U3', 'A'], ['U3', 'D']]


In [19]:
g.summary(1).splitlines()[-1]

'U1->A, U1->C, A->B, C->D, B->C, U3->A, U3->D'